In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/pre-processed-raw.csv')
# Drop unnecessary columns
df_clean = df.drop(columns=['date', 'credit_card', 'city', 'state', 'zipcode', 'credit_card_limit', 'transaction_dollar_amount', 'Long', 'Lat'])

# Check if the dataframe has any NaN or infinite values
if df_clean.isnull().values.any():
    df_clean.fillna(df_clean.mean(), inplace=True)

if np.isinf(df_clean.values).any():
    df_clean = df_clean.replace([np.inf, -np.inf], np.nan)
    df_clean.fillna(df_clean.mean(), inplace=True)

df_array = np.array(df_clean)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df_array)
df_scaled = scaler.transform(df_array)
df_scaled += 1e-10

# Check for any remaining NaN values
if np.isnan(df_scaled).any():
    df_scaled = np.nan_to_num(df_scaled)

/home/aym-workstation/anaconda3/envs/py3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/aym-workstation/anaconda3/envs/py3.7/lib/python3.7/site-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction ** 2 / new_sample_count
/home/aym-workstation/anaconda3/envs/py3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [4]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Define the dimension of our autoencoder
input_dim = df_scaled.shape[1]
encoding_dim = int(input_dim / 2)

# Define the autoencoder layers
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="tanh")(input_layer)
decoder = Dense(input_dim, activation="relu")(encoder)

# Define the autoencoder model
autoencoder = Model(inputs=input_layer, outputs=decoder)

# Compile the model
autoencoder.compile(optimizer=Adam(lr=0.001), loss="mean_squared_error")

# Train the autoencoder
history = autoencoder.fit(df_scaled, df_scaled, epochs=100, batch_size=32, shuffle=True, validation_split=0.2)

# Use the autoencoder to reconstruct the input data
reconstructed_data = autoencoder.predict(df_scaled)

# Calculate the mean squared error for each instance
mse = np.mean(np.power(df_scaled - reconstructed_data, 2), axis=1)

# Define a threshold
mse_threshold = np.quantile(mse, 0.95)  # consider as anomalies the 5% of the instances with highest mse

# Add the anomaly prediction (anomaly=1, normal=0) to the dataframe
df['anomaly'] = [1 if error > mse_threshold else 0 for error in mse]

# Get anomalies
anomalies = df[df['anomaly']==1]
print(anomalies)

2023-06-18 22:41:27.550841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 22:41:30.362960: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 22:41:30.364320: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
/home/aym-workstation/anaconda3/envs/py3.7/lib/python3.7/site-packages/keras/

Epoch 1/100
7365/7365 [==============================] - 37s 5ms/step - loss: 0.5357 - val_loss: 0.5248
Epoch 2/100
7365/7365 [==============================] - 30s 4ms/step - loss: 0.4698 - val_loss: 0.5087
Epoch 3/100
7365/7365 [==============================] - 31s 4ms/step - loss: 0.4599 - val_loss: 0.5002
Epoch 4/100
7365/7365 [==============================] - 35s 5ms/step - loss: 0.4557 - val_loss: 0.4975
Epoch 5/100
7365/7365 [==============================] - 31s 4ms/step - loss: 0.4536 - val_loss: 0.4955
Epoch 6/100
7365/7365 [==============================] - 39s 5ms/step - loss: 0.4522 - val_loss: 0.4950
Epoch 7/100
7365/7365 [==============================] - 31s 4ms/step - loss: 0.4512 - val_loss: 0.4916
Epoch 8/100
7365/7365 [==============================] - 41s 6ms/step - loss: 0.4505 - val_loss: 0.4936
Epoch 9/100
7365/7365 [==============================] - 36s 5ms/step - loss: 0.4500 - val_loss: 0.4908
Epoch 10/100
7365/7365 [==============================] - 36s 5m

In [6]:
len(df), len(anomalies)

(294588, 14730)